In [ ]:
import glob
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import os
from unyt import uvstack
import yt
from yt.utilities.physical_constants import G
from yt.visualization.color_maps import yt_colormaps

from yt.extensions.p2p.data_plotter import DataPlotter
from yt.extensions.p2p.models import \
    find_peaks, \
    rebin_profiles, \
    get_datasets

In [ ]:
pyplot.rcParams['figure.figsize'] = (15, 9)
pyplot.rcParams['font.size'] = 20

In [ ]:
yt.mylog.setLevel(40)

## Choose a star

In [ ]:
star_id = 334267081  ### first star
# star_id = 334267082  ### irradiated 1x, no heating, metal-free
# star_id = 334267083  ### irradiated 2x, heated 1x, metal-enriched (original target halo)
# star_id = 334267086  ### irradiated 2x, no heating, metal-free
# star_id = 334267090  ### irradiated 3x, heated 2x, metal-free
# star_id = 334267093  ### irradiated 4x, heated 2x, metal-free
# star_id = 334267099  ### irradiated 6x, minimal signs of heating, metal-free
# star_id = 334267102  ### irradiated 7x, heated 1x, metal-free
# star_id = 334267111  ### (target halo) irradiated 8x, multiple heatings, metal-enriched, some centering issues

In [ ]:
data_dir = f'star_minihalo_profiles/star_{star_id}'

## Load all profiles

In [ ]:
nfns = sorted(glob.glob(os.path.join(data_dir, '*_weight_field_None.h5')))
npds_list = get_datasets(nfns)
vfns = sorted(glob.glob(os.path.join(data_dir, '*_weight_field_volume.h5')))
vpds_list = get_datasets(vfns)
mfns = sorted(glob.glob(os.path.join(data_dir, '*_weight_field_mass.h5')))
mpds_list = get_datasets(mfns)

## Plot a bunch of profiles

Change `x_field` below to one of the other values to plot profiles vs. enclosed gas or total mass.

In [ ]:
x_field = 'radius'
# x_field = 'm_gas_enc'
# x_field = 'm_tot_enc'

plots = (
    ('gas_density', x_field),
    ('dm_density', x_field),
    ('temperature', x_field),
    ('entropy', x_field),
    ('pressure', x_field),
    ('hyd_eq_pressure', x_field),
    ('H2_fraction', x_field),
    ('metallicity', x_field),
    ('m_gas_enc', 'radius'),
    ('baryon_fraction', x_field),
    ('mbe_ratio', x_field),
    ('timescale_ratio', x_field),
)

labels = {
    'radius': 'r [pc]',
    'm_gas_enc': 'M$_{gas,enc}$ [Msun]',
    'm_tot_enc': 'M$_{tot,enc}$ [Msun]',
    'gas_density': '$\\rho_{b}\ [g/cm^{3}]$',
    'dm_density': '$\\rho_{dm}\ [g/cm^{3}]$',
    'temperature': 'T [K]',
    'entropy': 'entropy [erg cm$^{2}$]',
    'pressure': 'p [dyne/cm$^{2}$]',
    'hyd_eq_pressure': 'p / p$_{eq}$',
    'H2_fraction': 'f$_{H2}$',
    'metallicity': 'Z [Zsun]',
    'baryon_fraction': 'M$_{gas,enc}$ / M$_{tot,enc}$',
    'mbe_ratio': 'M$_{gas,enc}$ / M$_{BE}$',
    'timescale_ratio': 't$_{cs}$ / t$_{d}$',
}

In [ ]:
### Bonnor-Ebert Mass constant
a = 1.67
b = (225 / (32 * np.sqrt(5 * np.pi))) * a**-1.5

n_plots = len(plots)
profile_data = []

plotter = DataPlotter(plots, labels)
pbar = yt.get_pbar("Plotting", len(npds_list)-1)

for i, (npds, vpds, mpds) in enumerate(zip(npds_list, vpds_list, mpds_list)):
    pbar.update(i)
    
    # Omit last profile because star has formed.
    if i == len(npds_list) - 1:
        continue

    x_bins = mpds.profile.x_bins

    used = mpds.data['data', 'used'].d.astype(bool)
    m_tot = npds.profile['data', 'matter_mass'].to('Msun')[used]
    m_tot_enc = m_tot.cumsum()
    m_gas = npds.profile['data', 'cell_mass'].to('Msun')[used]
    m_gas_enc = m_gas.cumsum()

    r = mpds.data['data', 'radius'][used].to('pc')
    rho = mpds.data['data', 'density'][used]
    T = mpds.data['data', 'temperature'][used]
    H2 = mpds.data['data', 'H2_p0_fraction'][used]
    Z = mpds.data['data', 'metallicity3'][used].to('Zsun')
    p = mpds.data['data', 'pressure'][used]
    entropy = mpds.data['data', 'entropy'][used].to('erg*cm**2')

    r_v = vpds.data['data', 'radius'][used]
    rho_v = vpds.data['data', 'density'][used]
    rho_dm = vpds.data['data', 'dark_matter_density'][used]
    rho_total = vpds.data['data', 'matter_density'][used]
    volume_weight = vpds.data['data', 'weight'][used]

    dr = np.diff(x_bins)[used]
    dp = (G * m_tot_enc * rho_v * dr / r**2)
    p_eq = dp[::-1].cumsum()[::-1].in_cgs()

    cs = mpds.data['data', 'sound_speed'][used]
    m_BE = (b * (cs**4 / G**1.5) * p**-0.5).to('Msun')
    m_rat = m_gas_enc / m_BE

    # t_ff = mpds.data['data', 'total_dynamical_time'][used].to('Myr') / np.sqrt(2)
    t_ff = mpds.data['data', 'dynamical_time'][used].to('Myr') / np.sqrt(2)
    t_cool = mpds.data['data', 'cooling_time'][used].to('Myr')
    t_cs = (2 * r / mpds.data['data', 'sound_speed'][used]).to('Myr')    
    t_rat = t_cs / t_ff
    # t_rat = t_cool / t_ff
    
    plot_data = {
        'radius': r,
        'm_gas_enc': m_gas_enc,
        'm_tot_enc': m_tot_enc,
        'gas_density': rho,
        'dm_density': rho_dm,
        'temperature': T,
        'entropy': entropy,
        'pressure': p,
        'hyd_eq_pressure': (p/p_eq), 
        'H2_fraction': H2,
        'metallicity': Z,
        'mbe_ratio': m_rat,
        'timescale_ratio': t_rat,
        'baryon_fraction': (m_gas_enc/m_tot_enc),
    }

    profile_datum = {"m_gas_enc": m_gas_enc, "m_tot_enc": m_tot_enc,
                     "m_rat": m_rat, "t_cs": t_cs, "external_pressure": p_eq}
    pfields = [field for field in mpds.field_list
               if field[0] == 'data' and field[1] not in ['x', 'x_bins']]
    profile_datum.update(dict((field, mpds.data[field][used]) for field in pfields))
    profile_data.append(profile_datum)

    color = pyplot.cm.turbo(float(i/(len(mfns)-1)))
    plotter.plot_data(plot_data, color=color)

pbar.finish()

plotter.plot_axes()
plotter.axes['mbe_ratio'].set_ylim(1e-4, 3)
plotter.axes['mbe_ratio'].axhline(y=1, color='black', linestyle='--')
plotter.axes['timescale_ratio'].axhline(y=1, color='black', linestyle='--')
pyplot.show()

## Rebin profiles by mass

This will rebin profiles by enclosed gas mass for use with the one-zone model.

In [ ]:
rebin_field = 'm_gas_enc'
cube_fn = f"star_minihalo_profile_cubes/star_{star_id}.h5"

In [ ]:
if not os.path.exists(cube_fn) or True:
    # rebin profiles by enclosed gas mass with bin density of 5 bins per dex
    new_profiles = rebin_profiles(profile_data, rebin_field, 5)
    
    # Smash rebinned profile data into 2D arrays so we can plot time evolution.
    profile_cube = dict((field, uvstack([new_prof[field] for new_prof in new_profiles]))
                        for field in new_profiles[0])
    
    ### Times for all datasets
    t_all = mpds.arr([ds.current_time.to('Myr') for ds in mpds_list[:-1]])
    t_all -= t_all[0]
    profile_cube["time"] = t_all
    
    yt.save_as_dataset(mpds_list[-1], filename=cube_fn, data=profile_cube)

In [ ]:
ds = yt.load(cube_fn)

time_index = -1
i_peaks  = find_peaks(ds, rebin_field, "m_rat", time_index)

plot = True
if plot:
    bin_data = ds.data['data', rebin_field][time_index]
    peak_data = ds.data['data', "m_rat"][time_index]
    peak_used = peak_data > 0

    pyplot.loglog(bin_data[peak_used], peak_data[peak_used])
    pyplot.scatter(bin_data[i_peaks], peak_data[i_peaks])
    pyplot.xlabel("M$_{gas, enc}$ [Msun]")
    pyplot.ylabel("peak field")
    pyplot.show()